In [2]:
import sys
import numpy as np
import torch
import learn2learn as l2l
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms

from PIL.Image import LANCZOS
from csv import writer

from PIL import Image
import pandas as pd

In [2]:
torch.cuda.is_available()

False

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2022.4.1021342353/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2022.4.1021342353/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python39.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [10]:
model = l2l.vision.models.get_pretrained_backbone(model='cnn4', dataset='mini-imagenet', root='../../dataset/backbones/', download=True,)

In [ ]:
MiniImageCNN()

In [20]:
a = train_tasks.sample()[0]
a.shape

torch.Size([2, 3, 84, 84])

In [26]:
b = backbone(a)
b.shape

torch.Size([2, 640])

In [1]:
b

NameError: name 'b' is not defined

In [11]:
model

CNN4Backbone(
  (0): ConvBlock(
    (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (normalize): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (1): ConvBlock(
    (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (normalize): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (2): ConvBlock(
    (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (normalize): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (3): ConvBlock(
    (max_pool): Ma

In [12]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(params)

113088


In [32]:
for p1, p2 in zip(model.parameters(), backbone.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        print('False')

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [17]:
backbone

ResNet12Backbone(
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (downsample): Sequential(
        (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (DropBlock): DropBlock()
    )


In [20]:
from data.taskers import gen_tasks
from src.zoo.archs import CCVAE, CEncoder, Classifier_VAE, ConvBase, ResNet12Backbone, WRN28Backbone, CDecoder, MiniImageCNN
from src.zoo.delpo_utils import setup, inner_adapt_delpo, loss, accuracy
from data.loaders import CUBirds200

In [40]:
model = MiniImageCNN(5,
             1,
             hidden_size=64,
             layers=4,
             channels=3,
             embedding_size=None)
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(params)

121093


In [41]:
40577 + 1601 + 121093

163271

In [8]:
CUBirds200(root='../../dataset/', mode='train', download=True)

In [4]:
image_trans = transforms.Compose([transforms.ToTensor()])

In [11]:
train_dataset = l2l.vision.datasets.CUBirds200(root='../../dataset', mode='test', transform=transforms.ToTensor())
train_dataset = l2l.data.MetaDataset(train_dataset)
trans = [
            l2l.data.transforms.FusedNWaysKShots(train_dataset,
                                                 n=5,
                                                 k=5 + 1),
            l2l.data.transforms.LoadData(train_dataset),
            l2l.data.transforms.RemapLabels(train_dataset),
            l2l.data.transforms.ConsecutiveLabels(train_dataset)
        ]
train_generator = l2l.data.TaskDataset(dataset=train_dataset, num_tasks=5)

In [12]:
for i, task in enumerate(train_generator):
    print(i, task)

TypeError: 'NoneType' object is not iterable

In [5]:
train_dataset = CUBirds200(root='../../dataset', mode='train', transform=image_trans)

In [6]:
dataset = l2l.data.MetaDataset(train_dataset)

In [7]:
trans = [
            l2l.data.transforms.FusedNWaysKShots(dataset,
                                                 n=5,
                                                 k=5 + 1),
            l2l.data.transforms.LoadData(dataset),
            l2l.data.transforms.RemapLabels(dataset),
            l2l.data.transforms.ConsecutiveLabels(dataset)
        ]
tasks = l2l.data.TaskDataset(dataset, task_transforms=trans)

In [8]:
tasks.sample()

RuntimeError: stack expects each tensor to be equal size, but got [3, 378, 500] at entry 0 and [3, 293, 200] at entry 1

In [4]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../dataset/mini_imagenet', mode='train', n_ways=5, k_shots=1, q_shots=1, image_transforms=None)

In [5]:
device = 'cpu'
n_ways = 5
k_shots = 5
q_shots = 10
dataset = 'mini_imagenet'
root = '../../dataset/mini_imagenet'
order = False
inner_lr = 0.001
meta_lr = 0.0001
reconst_loss = nn.MSELoss(reduction='none')
adapt_steps = 2
meta_batch_size = 10
download = False

In [15]:
class Args:
    def __init__(self, wt_ce, klwt, rec_wt, beta_l, beta_s):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.wt_ce = wt_ce
        self.klwt = klwt
        self.rec_wt = rec_wt
        self.beta_l = beta_l
        self.beta_s = beta_s
        self.dataset = 'miniimagenet'
        self.task_adapt = True
        self.task_adapt_fn = 'tafe'
        self.n_ways = 5
        self.k_shots = 1
        self.q_shots = 10
        self.backbone = ["False", '', '']
        self.device = 'cpu'
args = Args(100, False, 0.01, 1, 1)

In [24]:
import timm

In [28]:
model = timm.create_model('resnet18')
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(params)

11689512


In [21]:
learner = CCVAE(in_channels=3, base_channels=64, n_ways=n_ways, dataset='miniimagenet', task_adapt=True, task_adapt_fn=args.task_adapt_fn, args=args, latent_dim_l=64, latent_dim_s=64)

In [22]:
model_sub = learner.classifier_vae.encoder
params = sum(p.numel() for p in model_sub.parameters() if p.requires_grad)
print(params)

118752


In [23]:
model_sub = learner.encoder
params = sum(p.numel() for p in model_sub.parameters() if p.requires_grad)
print(params)

113088


In [11]:
35840 - 28896

6944

In [19]:
34560 - 28896

5664

In [12]:
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(learner, (3, 84, 84), as_strings=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

TypeError: forward() missing 1 required positional argument: 'update'

In [7]:
task = train_tasks.sample()
learner = CCVAE(in_channels=3, base_channels=32, n_ways=n_ways, dataset='miniimagenet', task_adapt=True, task_adapt_fn=args.task_adapt_fn, args=args)
learner = learner.to(device)
learner = l2l.algorithms.MAML(learner, first_order=order, lr=inner_lr, allow_nograd=False)


In [ ]:

model = learner.clone()
evaluation_loss, evaluation_accuracy, reconst_img, query_imgs, mu_l, log_var_l, mu_s, log_var_s, logits, labels = inner_adapt_delpo(
                task, reconst_loss, model, args.n_ways, args.k_shots, args.q_shots, adapt_steps, args.device, True, args)

In [20]:
labels.unsqueeze(dim=1).shape

torch.Size([5, 1])

In [23]:
a = torch.cat([torch.full((5,1), 1), logits, labels.unsqueeze(dim=1)], axis=1)
a = np.array(a)
a.shape

(5, 7)

In [24]:
a

array([[ 1.        , -0.06661633,  0.35360354, -0.20023935,  0.18052828,
        -0.09857406,  2.        ],
       [ 1.        ,  0.15822504,  0.3527588 ,  0.07243241,  0.11000247,
        -0.36092955,  4.        ],
       [ 1.        ,  0.2106584 ,  0.25372577,  0.22026452,  0.19547145,
        -0.05639004,  1.        ],
       [ 1.        , -0.16028145,  0.359845  ,  0.33969116,  0.2760064 ,
        -0.3392319 ,  0.        ],
       [ 1.        ,  0.21336251,  0.09490922, -0.04687326,  0.24869294,
        -0.18386593,  3.        ]], dtype=float32)

In [25]:
columns = ['task', 'accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss']
#columns = columns.append(additional)
df = pd.DataFrame(columns=columns)
df.to_csv('./temp.csv')

In [26]:
with open('./temp.csv', 'a+', newline='') as write_obj:
            # Create a writer object from csv module
            csv_writer = writer(write_obj)
            # Add contents of list as last row in the csv file
            csv_writer.writerows(a)
            

In [14]:
a = setup(args.dataset, root, n_ways, k_shots, q_shots, order, inner_lr, device, download, args.task_adapt, args.task_adapt_fn, args)

In [15]:
for p in a[-1].parameters():
    print(p.requires_grad)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [18]:
a[-1].allow_nograd

False

In [ ]:
for _ in range(adapt_steps):
        if args.task_adapt:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
                torch.cat([support, queries], dim=0), 'inner')
        else:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
                support, 'inner')
        adapt_loss = loss(reconst_loss, reconst_image, support,
                            logits, support_labels, mu_s, log_var_s, mu_l, log_var_l, args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s)
        learner.adapt(adapt_loss['elbo'])

    if args.task_adapt:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
            torch.cat([support, queries], dim=0), 'outer')
    else:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
            queries, 'outer')

    eval_loss = loss(reconst_loss, reconst_image, queries,
                     logits, queries_labels, mu_s, log_var_s, mu_l, log_var_l, args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s)
    eval_acc = accuracy(F.softmax(logits, dim=1), queries_labels)


In [25]:
backbone = WRN28Backbone()
weights = torch.load('../../dataset/backbones/pretrained_models/mini-imagenet/wrn28.pth', map_location='cpu')
backbone.load_state_dict(weights)
backbone.to('cpu')

WRN28Backbone(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer1): Sequential(
    (0): wide_basic(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout): Dropout(p=0.0, inplace=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (1): wide_basic(
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout): Dropout(p=0.0, inplace=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Con

In [31]:
x = backbone(train_tasks.sample()[0].to('cpu'))
x = x.reshape(x.shape[-1], x.shape[0], x.shape[1], -1)
x.shape

torch.Size([1, 30, 640, 1])

In [35]:
x.squeeze(dim=0).squeeze(dim=2).shape

torch.Size([30, 640])

In [8]:
encoder = CEncoder(num_input_channels=3, base_channel_size=32, latent_dim=64, dataset=dataset, args=args, flag=True)


In [9]:

decoder = CDecoder(num_input_channels=3, base_channel_size=32, latent_dim=(64 + 64), dataset=dataset)


In [11]:

classifier_vae = Classifier_VAE(3, 32, 64, 5, dataset, task_adapt=True, task_adapt_fn='eaen', args=args)


In [1]:
a = 0
b = 1
c = 2

if a == 1010:
    print('yes')
elif a == 0:
    if b == 1:
        print(c)
    elif b == 2:
        print('nonlocal')

2


In [8]:
class TADCEncoder(nn.Module):
    """ Convolutional Encoder to transform an input image into a latent-space gaussian distribution parametrized by its mean and log-variance, 
    with the added functionality of the distributions being task/episode aware. """

    def __init__(self,
                 num_input_channels: int,
                 base_channel_size: int,
                 latent_dim: int,
                 dataset: str,
                 task_adapt_fn: str,
                 args,
                 act_fn: object = nn.ReLU):
        """
        Inputs:
            - num_input_channels : Number of input channels of the image
            - base_channel_size : Number of channels we use in the first convolutional layers. Deeper layers use 2x of it.
            - latent_dim : Dimensionality of latent representation z
            - dataset: name of the dataset
            - task_adapt_fn: EAEN (eaen) or Kernel Smoothing (gks)
            - args: dict of arguments
            - act_fn : Activation function used throughout the encoder network
        """

        super(TADCEncoder, self).__init__()
        c_hid = base_channel_size
        self.args = args
        self.task_adapt_fn = task_adapt_fn

        if dataset == 'omniglot':
            self.net = nn.Sequential(
                nn.Conv2d(num_input_channels, c_hid,
                          kernel_size=3, padding=1, stride=(2, 2)),
                nn.BatchNorm2d(c_hid),
                act_fn(),  # 14

                nn.Conv2d(c_hid, c_hid, kernel_size=3,
                          padding=1, stride=(2, 2)),
                nn.BatchNorm2d(c_hid),
                act_fn(),  # 7

                nn.Conv2d(c_hid, c_hid, kernel_size=3,
                          padding=1, stride=(2, 2)),
                nn.BatchNorm2d(c_hid),
                act_fn(),  # 4

                nn.Conv2d(c_hid, c_hid, kernel_size=3,
                          padding=1, stride=(2, 2)),
                nn.BatchNorm2d(c_hid),
                act_fn()  # 2
            )
            self.h1 = nn.Linear(4*c_hid, latent_dim)
            self.h2 = nn.Linear(4*c_hid, latent_dim)

        elif (dataset == 'mini_imagenet') or (dataset == 'cifarfs') or (dataset == 'tiered'):
            if args.pretrained[0] == True:
                self.h1 = nn.Sequential(nn.Linear(
                    640*25, 8000), nn.Linear(8000, 1000), nn.Linear(1000, latent_dim))
                self.h2 = nn.Sequential(nn.Linear(
                    640*25, 8000), nn.Linear(8000, 1000), nn.Linear(1000, latent_dim))

            elif args.pre_trained[0] == False:
                self.net = nn.Sequential(
                    nn.Conv2d(num_input_channels, c_hid,
                              kernel_size=3, padding=1),
                    nn.BatchNorm2d(c_hid),
                    act_fn(),
                    nn.MaxPool2d(2),  # 28 x 28, # 42 x 42

                    # nn.ZeroPad2d(conv_padding),
                    nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
                    nn.BatchNorm2d(c_hid),
                    act_fn(),
                    nn.MaxPool2d(2),  # 9x9 # 21 x 21

                    # nn.ZeroPad2d(conv_padding),
                    nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
                    nn.BatchNorm2d(c_hid),
                    act_fn(),
                    nn.MaxPool2d(2),  # 3x3 # 10 x 10

                    # nn.ZeroPad2d(conv_padding),
                    nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
                    nn.BatchNorm2d(c_hid),
                    act_fn(),
                    nn.MaxPool2d(2)  # 1x1 # 5 x 5
                )

                if (dataset == 'mini_imagenet') or (dataset == 'tiered'):
                    self.h1 = nn.Linear(c_hid*25, latent_dim)
                    self.h2 = nn.Linear(c_hid*25, latent_dim)
                elif dataset == 'cifarfs':
                    self.h1 = nn.Linear(c_hid*4, latent_dim)
                    self.h2 = nn.Linear(c_hid*4, latent_dim)

        self.n = args.n_ways * (args.k_shots + args.q_shots)
        self.eaen = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(
                self.n, 1), stride=(1, 1), padding='valid', bias=False),
            # P
            act_fn(),

            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(
                1, 1), stride=(1, 1), padding='valid', bias=False),
            # Z
            act_fn(),

            nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(
                1, 1), stride=(1, 1), padding='valid', bias=False),
            # F
            act_fn(),
        )

    def forward(self, x, update: str):
        if self.args.pretrained[0] == False:
            x = self.net(x)
        elif self.args.pretrained[0] == True:
            x = x

        # Task aware embeddings

        if self.task_adapt_fn == 'eaen':
            G = x.permute(2, 3, 0, 1)
            G = G.reshape(G.shape[0] * G.shape[1],
                          G.shape[2], G.shape[3]).unsqueeze(dim=1)
            G = self.eaen(G)
            G = G.squeeze().transpose(0, 1).reshape(-1, x.shape[2], x.shape[3])
            if update == 'inner':
                x = x[:self.args.n_ways*self.args.k_shots] * G
            elif update == 'outer':
                x = x[self.args.n_ways*self.args.k_shots:] * G
            x = nn.Flatten()(x)

        elif self.task_adapt_fn == 'gks':
            G = x.reshape(self.n, -1)
            A = torch.cdist(G, G, p=2) ** 2
            A = A / A.var()  # Normalized adjacency matrix
            D = torch.diag(A.sum(dim=1).pow(-0.5))
            L = torch.matmul(torch.matmul(D, A), D)  # Laplacian Matrix
            I = torch.eye(self.n, self.n).to(self.args.device)
            P = torch.linalg.inv(I - self.args.alpha * L)  # Propagator Matrix
            x = torch.matmul(P, G)
            if update == 'inner':
                x = x[:self.args.n_ways*self.args.k_shots]
            elif update == 'outer':
                x = x[self.args.n_ways*self.args.k_shots:]

        mu = self.h1(x)
        log_var = self.h2(x)
        return mu, log_var


In [9]:
tadcen = TADCEncoder(3, 32, 64, 'mini_imagenet', 'eaen', args)

In [18]:
classifier_vae

Classifier_VAE(
  (encoder): TADCEncoder(
    (eaen): Sequential(
      (0): Conv2d(1, 64, kernel_size=(30, 1), stride=(1, 1), padding=valid, bias=False)
      (1): ReLU()
      (2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=valid, bias=False)
      (3): ReLU()
      (4): Conv2d(32, 1, kernel_size=(1, 1), stride=(1, 1), padding=valid, bias=False)
      (5): ReLU()
    )
  )
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=5, bias=True)
  )
)

In [17]:
learner = CCVAE(in_channels=3, base_channels=32, n_ways=n_ways, dataset='mini_imagenet', task_adapt=args.task_adapt, task_adapt_fn=args.task_adapt_fn, args=args)

KeyboardInterrupt: 

In [ ]:
learner = learner.to(device)
learner = l2l.algorithms.MAML(learner, first_order=order, lr=inner_lr)

In [9]:

# Init the Backbone
if args.pretrained[0] == True:
    backbone = ResNet12Backbone(
        args, avg_pool=True if args.pretrained[2] == 640 else False)  # F => 16000; T => 640
    weights = torch.load(args.pretrained[1], map_location='cpu')
    backbone.load_state_dict(weights)
    backbone.to(device)
    # Freeze the backbone
    for p in backbone.parameters():
        p.requires_grad = False
else:
    backbone = 'None'


In [10]:

def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def kl_div(mus, log_vars):
    return - 0.5 * (1 + log_vars - mus**2 - torch.exp(log_vars)).sum(dim=1)


def loss(reconst_loss: object, reconst_image, image, logits, labels, mu_s, log_var_s, mu_l, log_var_l, wt_ce=1e2, klwt=False, rec_wt=1e-2, beta_l=1, beta_s=1):
    kl_div_s = kl_div(mu_s, log_var_s).mean()
    kl_div_l = kl_div(mu_l, log_var_l).mean()
    if klwt:
        kl_wt = mu_s.shape[-1] / (image.shape[-1] *
                                  image.shape[-2] * image.shape[-3])
    else:
        kl_wt = 1

    ce_loss = torch.nn.CrossEntropyLoss()
    classification_loss = ce_loss(logits, labels)
    rec_loss = reconst_loss(reconst_image, image)
    rec_loss = rec_loss.view(rec_loss.shape[0], -1).sum(dim=-1).mean()

    L = wt_ce*classification_loss + beta_l*kl_wt*kl_div_l + \
        rec_wt*rec_loss + beta_s*kl_wt*kl_div_s  # -log p(x,y)

    losses = {'elbo': L, 'label_kl': kl_div_l, 'style_kl': kl_div_s,
              'reconstruction_loss': rec_loss, 'classification_loss': classification_loss}

    return losses


In [11]:
task = train_tasks.sample()

In [12]:
data, labels = task
if args.dataset == 'miniimagenet':
    data, labels = data.to(device) / 255.0, labels.to(device)
elif (args.dataset == 'omniglot') or (args.dataset == 'cifarfs') or (args.dataset == 'tiered'):
    data, labels = data.to(device), labels.to(device)
total = n_ways * (k_shots + q_shots)
queries_index = np.zeros(total)

# Extracting the evaluation datums from the entire task set, for the meta gradient calculation
for offset in range(n_ways):
    queries_index[np.random.choice(
        k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]


In [13]:
queries.shape

torch.Size([25, 3, 84, 84])

In [14]:

if args.pretrained[0] == True:
    support_ext = backbone(support)
    queries_ext = backbone(queries)


/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [15]:
support_ext.shape

torch.Size([5, 640, 5, 5])

In [16]:

# Inner adapt step
if args.pretrained[0] == True:
    for _ in range(adapt_steps):
        if args.task_adapt:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner([torch.cat(
                [support_ext, queries_ext], dim=0), torch.cat([support, queries], dim=0)], 'inner')
        else:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
                [support_ext, support], 'inner')
        adapt_loss = loss(reconst_loss, reconst_image, support,
                            logits, support_labels, mu_s, log_var_s, mu_l, log_var_l, args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s)
        learner.adapt(adapt_loss['elbo'])

    if args.task_adapt:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner([torch.cat(
            [support_ext, queries_ext], dim=0), torch.cat([support, queries], dim=0)], 'outer')
    else:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
            [queries_ext, queries], 'outer')

if args.pretrained[0] == False:
    for _ in range(adapt_steps):
        if args.task_adapt:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
                torch.cat([support, queries], dim=0), 'inner')
        else:
            reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
                support, 'inner')
        adapt_loss = loss(reconst_loss, reconst_image, support,
                            logits, support_labels, mu_s, log_var_s, mu_l, log_var_l, args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s)
        learner.adapt(adapt_loss['elbo'])

    if args.task_adapt:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
            torch.cat([support, queries], dim=0), 'outer')
    else:
        reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(
            queries, 'outer')

eval_loss = loss(reconst_loss, reconst_image, queries,
                    logits, queries_labels, mu_s, log_var_s, mu_l, log_var_l, args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s)
eval_acc = accuracy(F.softmax(logits, dim=1), queries_labels)


AttributeError: 'TADCEncoder' object has no attribute 'h1'

In [7]:
backbone = ResNet12Backbone(args=args, avg_pool=True)
weights = torch.load('../../dataset/backbones/mini/25u1gtp5.pth', map_location='cpu')
backbone.load_state_dict(weights)
backbone.to('cuda')

ResNet12Backbone(
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (downsample): Sequential(
        (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (DropBlock): DropBlock()
    )


In [8]:
for p in backbone.parameters():
    p.requires_grad = False

In [10]:
x = backbone(train_tasks.sample()[0].to('cuda'))
x.shape

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([100, 640])

In [21]:
#x.detach()
x.requires_grad

False